# Setup 

In [10]:
import os, requests, openpyxl 
import pandas as pd 
from io import BytesIO 

from azure.identity import ClientSecretCredential 


# read in the credentials 
tenant_id = os.environ.get("AZ_TENANT_ID") 
client_id = os.environ.get("AZ_CLIENT_ID") 
client_secret = os.environ.get("AZ_CLIENT_SECRET") 

## Parameters 

In [3]:
# file parameters 
host_name = "dailydataapps" 
site_name = "SharePointDemo" 
drive_name = "Documents" 
file_name = "Demo Excel Data.xlsx" 

# file paths 
sharepoint_file = f"General//{file_name}" 

# base URL for the Microsoft Graph API 
base_url = "https://graph.microsoft.com" 

## Access Token 

In [4]:
# generate an access token 
scopes = [f"{base_url}/.default"] 
credentials = ClientSecretCredential(tenant_id, client_id, client_secret) 
access_token = credentials.get_token(*scopes).token 

# add the token to a headers dictionary 
headers = {
    "Authorization": f"Bearer {access_token}" 
} 

## Site and Drive ID 

In [ ]:
# Get the SharePoint site ID 
site_url = f"{base_url}/v1.0/sites/{host_name}.sharepoint.com:/sites/{site_name}"
site_response = requests.get(site_url, headers = headers)  

# get the site ID 
site_id = site_response.json()['id'] 

# Get the drive ID
drive_url = f"{base_url}/v1.0/sites/{site_id}/drives"
drive_response = requests.get(drive_url, headers = headers)
drive_id = drive_response.json()['value'][0]['id'] 

# File Download 

In [6]:
# Define the spreadsheet URL
file_url = f"{base_url}/v1.0/sites/{site_id}/drives/{drive_id}/root:/{sharepoint_file}:/content" 

# make the API request 
response = requests.get(file_url, headers = headers)

# check if the request was successful 
if response.status_code == 200:

    # if successful, load the workbook with openpyxl 
    file = BytesIO(response.content) 
    wb = openpyxl.load_workbook(file) 

    # update the user 
    print("File downloaded successfully.") 

File downloaded successfully.


## Import as Dataframe 

In [ ]:
# read in the table by name 
sheet = wb["Sheet1"] 
data = sheet.tables["demo_table"] 

# get the table range and extract values
table_range = sheet[data.ref]

# extract all values from the table range
table_data = []
for row in table_range:
    table_data.append([cell.value for cell in row])

# convert to dataframe with first row as headers
df = pd.DataFrame(table_data[1:], columns = table_data[0])

# showcase the data 
df 

,Col1,Col2,Col3
0,1,5.3,87
1,3,4.2,53
2,2,9.1,254
